In [37]:
import numpy as np
import pandas as pd
import subprocess
import re

In [53]:
%%bash
source ~/.bashrc
conda activate ipyrad
bcftools query -l ../apal_coords_snpchip_conserved_probes.recode.vcf.gz \
    > ../microarray_samples.txt

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


In [24]:
%%bash
source ~/.bashrc
conda activate ipyrad
bcftools query -l ../calls/all.phased.tagged.bcf > ../panelsample.txt

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


To generate a list of genets to impute for the GWAS, we'll find unique genets that were phenotyped but don't have existing WGS data:

In [123]:
galmet = pd.read_csv("../galaxy_metadata.txt", sep = "\t")

In [124]:
metadata = pd.read_csv("../2024_01_12_PsuData_fixed.csv")
metadata = metadata.reset_index(drop=False)
metadata.columns = ['GWAS_name', 'genetNumber', 'FixedTubeID', 'cbassRun', 'species', 'Notes',
       'Novogene_ID', 'geno', 'Best Galaxy Match Reef Name',
       'Best Galaxy Match Clonal ID', 'Fixed Best Galaxy Match Clonal ID',
       'Best Galaxy Match KING Coefficient', 'Questionable?', 'presumed_geno',
       'name', 'year', 'nursery', 'region', 'fixed_ed50', 'ed50', 'stdError',
       'nurseRes', 'ed50Adj', 'temperature', 'fixed_logSh', 'logSh', 'dom',
       'Best Galaxy Match', 'Best Galaxy Match User ID']
metadata["GWAS_name"] = "sample" + metadata["GWAS_name"].astype(str) 

In [125]:
microarrayvcfsamples = pd.read_csv("../microarray_samples.txt", header = None, names = ["ID"])

In [126]:
galmet = galmet[galmet["Affymetrix ID"].isin(microarrayvcfsamples["ID"])]

In [127]:
impute = metadata[metadata["Novogene_ID"].isna()]

In [128]:
impute = impute[impute["Fixed Best Galaxy Match Clonal ID"].notna()].reset_index(drop=True)

In [129]:
impute["impute_samp"] = np.nan
for i in range(len(impute)):
    tmp = (galmet[galmet["Coral Mlg Clonal ID"] == impute["Fixed Best Galaxy Match Clonal ID"][i]]
                 .sort_values("Percent Missing Data Coral")
                 .reset_index(drop=True))
    impute["impute_samp"][i] = tmp["Affymetrix ID"].head(1)[0]

/tmp/ipykernel_2402507/3397860949.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impute["impute_samp"][i] = tmp["Affymetrix ID"].head(1)[0]


In [133]:
for i in range(len(impute)):
    task = "subset_vcf"
    mem = "16"
    cpus = "1"
    infile = "../apal_coords_snpchip_conserved_probes.recode.vcf.gz"
    outpath = "~/scratch"
    samp = impute["impute_samp"][i].replace("(", "\(").replace(")", "\)")
    gwas_id = str(impute["GWAS_name"][i])
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         infile + " " + outpath + " " + samp + " " + gwas_id], shell=True)

Submitted batch job 9028141
Submitted batch job 9028142
Submitted batch job 9028143
Submitted batch job 9028144
Submitted batch job 9028145
Submitted batch job 9028146
Submitted batch job 9028147
Submitted batch job 9028148
Submitted batch job 9028149
Submitted batch job 9028150
Submitted batch job 9028151
Submitted batch job 9028152
Submitted batch job 9028153
Submitted batch job 9028154
Submitted batch job 9028155
Submitted batch job 9028156
Submitted batch job 9028157
Submitted batch job 9028158
Submitted batch job 9028159
Submitted batch job 9028160
Submitted batch job 9028161
Submitted batch job 9028162
Submitted batch job 9028163
Submitted batch job 9028164
Submitted batch job 9028165
Submitted batch job 9028166
Submitted batch job 9028167
Submitted batch job 9028168
Submitted batch job 9028169
Submitted batch job 9028170
Submitted batch job 9028171
Submitted batch job 9028172
Submitted batch job 9028173
Submitted batch job 9028174
Submitted batch job 9028175
Submitted batch job 

In [134]:
for i in impute["GWAS_name"].tolist():
    (impute[["impute_samp", "GWAS_name"]]
     [impute["GWAS_name"] == i].set_index("impute_samp")
     .to_csv("~/scratch/" + str(i) + "_rename.txt", sep = " ", header = None))

In [135]:
for i in range(len(impute)):
    task = "reheader"
    mem = "16"
    cpus = "1"
    infile = "/scratch/nsl5160/" + str(impute["GWAS_name"][i]) + ".vcf.gz"
    outfile = "/scratch/nsl5160/" + str(impute["GWAS_name"][i]) + "_reheaded.vcf.gz"
    renamefile = '/scratch/nsl5160/' + str(impute["GWAS_name"][i]) + "_rename.txt"
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         infile + " " + outfile + " " + renamefile], shell=True)

Submitted batch job 9028269
Submitted batch job 9028270
Submitted batch job 9028271
Submitted batch job 9028272
Submitted batch job 9028273
Submitted batch job 9028274
Submitted batch job 9028275
Submitted batch job 9028276
Submitted batch job 9028277
Submitted batch job 9028278
Submitted batch job 9028279
Submitted batch job 9028280
Submitted batch job 9028281
Submitted batch job 9028282
Submitted batch job 9028283
Submitted batch job 9028284
Submitted batch job 9028285
Submitted batch job 9028286
Submitted batch job 9028287
Submitted batch job 9028288
Submitted batch job 9028289
Submitted batch job 9028290
Submitted batch job 9028291
Submitted batch job 9028292
Submitted batch job 9028293
Submitted batch job 9028294
Submitted batch job 9028295
Submitted batch job 9028296
Submitted batch job 9028297
Submitted batch job 9028298
Submitted batch job 9028299
Submitted batch job 9028300
Submitted batch job 9028301
Submitted batch job 9028302
Submitted batch job 9028303
Submitted batch job 

In [136]:
%%bash
source ~/.bashrc
conda activate beagle
bcftools merge /scratch/nsl5160/sample*_reheaded.vcf.gz -O z \
    > ../calls/phenotyped_microarray.vcf.gz

In [137]:
%%bash
source ~/.bashrc
conda activate ipyrad
rm ../calls/all.phased.tagged.vcf.gz.tbi
bcftools view ../calls/all.phased.tagged.bcf -O z > ../calls/all.phased.tagged.vcf.gz
tabix ../calls/all.phased.tagged.vcf.gz

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


In [138]:
%%bash
source ~/.bashrc
conda activate beagle
rm /scratch/nsl5160/phenotyped_microarray.vcf.gz.tbi
zcat ../calls/phenotyped_microarray.vcf.gz | sed "s/C,C/C/g" \
    > /scratch/nsl5160/phenotyped_microarray.vcf
rm /scratch/nsl5160/phenotyped_microarray.vcf.gz
bgzip /scratch/nsl5160/phenotyped_microarray.vcf
tabix /scratch/nsl5160/phenotyped_microarray.vcf.gz

In [141]:
for chrs in ['Apal_hic_scaffold_10',
 'Apal_hic_scaffold_35',
 'Apal_hic_scaffold_2',
 'Apal_hic_scaffold_30',
 'Apal_hic_scaffold_20',
 'Apal_hic_scaffold_17',
 'Apal_hic_scaffold_4',
 'Apal_hic_scaffold_31',
 'Apal_hic_scaffold_5',
 'Apal_hic_scaffold_15',
 'Apal_hic_scaffold_6',
 'Apal_hic_scaffold_1',
 'Apal_hic_scaffold_21',
 'Apal_hic_scaffold_11']:
    task = "beagle_impute"
    mem = "128"
    cpus = "20"
    inpath = "/scratch/nsl5160/"
    prefix = "phenotyped_microarray"
    outpath = "/scratch/nsl5160/"
    refpanel = "../calls/all.phased.tagged.vcf.gz"
    ne = 10000
    chrs = chrs
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         inpath + " " + prefix + " " + outpath + " " + refpanel + " " + str(ne) + " " + chrs], shell=True)

Submitted batch job 9028399
Submitted batch job 9028400
Submitted batch job 9028401
Submitted batch job 9028402
Submitted batch job 9028403
Submitted batch job 9028404
Submitted batch job 9028405
Submitted batch job 9028406
Submitted batch job 9028407
Submitted batch job 9028408
Submitted batch job 9028409
Submitted batch job 9028410
Submitted batch job 9028411
Submitted batch job 9028412


In [142]:
%%bash
source ~/.bashrc
conda activate beagle
rm ../calls/phenotyped_microarray_imputed.vcf.gz.tbi
for i in $( ls /scratch/nsl5160/phenotyped_microarray_*_imputed.vcf.gz ); do
    tabix ${i}
done
bcftools concat /scratch/nsl5160/phenotyped_microarray_*_imputed.vcf.gz \
    -O z > ../calls/phenotyped_microarray_imputed.vcf.gz 
tabix ../calls/phenotyped_microarray_imputed.vcf.gz 

Checking the headers and starting positions of 14 files
Concatenating /scratch/nsl5160/phenotyped_microarray_Apal_hic_scaffold_10_imputed.vcf.gz	9.352291 seconds
Concatenating /scratch/nsl5160/phenotyped_microarray_Apal_hic_scaffold_11_imputed.vcf.gz	6.242336 seconds
Concatenating /scratch/nsl5160/phenotyped_microarray_Apal_hic_scaffold_15_imputed.vcf.gz	7.356026 seconds
Concatenating /scratch/nsl5160/phenotyped_microarray_Apal_hic_scaffold_17_imputed.vcf.gz	8.446682 seconds
Concatenating /scratch/nsl5160/phenotyped_microarray_Apal_hic_scaffold_1_imputed.vcf.gz	6.947039 seconds
Concatenating /scratch/nsl5160/phenotyped_microarray_Apal_hic_scaffold_20_imputed.vcf.gz	8.277677 seconds
Concatenating /scratch/nsl5160/phenotyped_microarray_Apal_hic_scaffold_21_imputed.vcf.gz	6.861835 seconds
Concatenating /scratch/nsl5160/phenotyped_microarray_Apal_hic_scaffold_2_imputed.vcf.gz	8.598096 seconds
Concatenating /scratch/nsl5160/phenotyped_microarray_Apal_hic_scaffold_30_imputed.vcf.gz	5.874262 

In [145]:
%%bash
source ~/.bashrc
conda activate beagle
rm ../calls/phenotyped_microarray_imputed_filteredDR2.vcf.gz.tbi
bcftools filter -e 'INFO/DR2<0.8' ../calls/phenotyped_microarray_imputed.vcf.gz \
    -O z > ../calls/phenotyped_microarray_imputed_filteredDR2.vcf.gz
tabix ../calls/phenotyped_microarray_imputed_filteredDR2.vcf.gz